# Shakespearean Text Generation with Hugging Face Transformers

This notebook shows how to load a pretrained causal language model from the Hugging Face Hub and use its `generate()` method to craft text that mimics Shakespeare's diction. Feel free to adapt the prompts and decoding parameters to experiment with different styles.


In [24]:
%pip install --quiet transformers torch accelerate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


## Load a lightweight Shakespeare-friendly model

The `distilgpt2` checkpoint is a distilled version of GPT-2 that fits on most laptops while still producing coherent prose. For better results you can swap in larger checkpoints (e.g., `gpt2-medium`, `tiiuae/falcon-7b-instruct`) if your hardware allows it.


In [26]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

MODEL_NAME = "tiiuae/falcon-7b-instruct"

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {DEVICE}")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
model.to(DEVICE)
model.eval();


Usando dispositivo: cpu


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model-00001-of-00002.safetensors:  92%|#########1| 9.13G/9.95G [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [ ]:
def generate_shakespearean_text(
    prompt: str,
    max_new_tokens: int = 120,
    temperature: float = 0.95,
    top_p: float = 0.92,
    repetition_penalty: float = 1.05,
    seed: int | None = 42,
) -> str:
    """Generate Shakespearean-style text using nucleus sampling."""
    if seed is not None:
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(seed)

    inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            repetition_penalty=repetition_penalty,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return generated_text


## Try a few Shakespearean prompts

Adjust the prompt and decoding hyperparameters to steer tone, length, and creativity. Lower `temperature` or `top_p` for more conservative prose; raise them for adventurous language.


In [ ]:
prompt = (
    "Write a Shakespearean sonnet with 14 lines and iambic pentameter"
)

sample = generate_shakespearean_text(
    prompt,
    max_new_tokens=125,
    temperature=1.2,
    top_p=0.9,
    repetition_penalty=1.03,
)
print(sample)


Write a Shakespearean sonnet with 14 lines and iambic pentameter:

He sings at three-o'clock, to see what's out of place
.. The King is dead, that he might live; He must not go through town or country Again To write this poem, then again It'll be very interesting if I were able to translate it for you All the times since Henry IV died When would his grief have filled me? If in your time there was no hope but God That any good may come back to him Who could endure so long an age on earth So much had life lived, such a lot had wroth And still we look upon nothing new as fresh
